In [1]:
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import pointbiserialr

sigmoid = lambda x: 1/(1 + np.exp(-x))

## Data generation

Note that $y$ is a function of both $x_1$ and $x_2$, and has not randomness, thus, it should be very straightforward and easy to predict it.


In [2]:
np.random.seed(42)

x1 = np.array([-1] * 1000 + [1] * 1000)
x2 = np.random.normal(size=2000)

y = (sigmoid((x1 * x2) * 3.4 + 1.2) > 0.5).astype(int)

df = pl.DataFrame({'x1': x1, 'x2': x2, 'y': y})

## Feature selection
A common approach is to evaluate correlation between predictors and the outcome.
Here, we see that neither of the predictors has any correlation with the outcome, so this approach would have missed both of them!


In [3]:
corr = pointbiserialr(x1, y)
print(f"x1 | y correlation: {corr.statistic:.3f} | p-value: {corr.pvalue:.3f}")

corr = pointbiserialr(x2, y)
print(f"x2 | y correlation: {corr.statistic:.3f} | p-value: {corr.pvalue:.3f}")

x1 | y correlation: 0.032 | p-value: 0.157
x2 | y correlation: -0.007 | p-value: 0.769


## Using a simple model

Even if we include all features in the model, a simple linear model might miss completely the interaction between $x_1$ and $x_2$!

In [4]:
train, test = train_test_split(df)

lr = LogisticRegression().fit(train.drop('y'), train['y'])

train = train.with_columns(y_pred = lr.predict_proba(train.drop('y'))[:, 1])
roc_auc = roc_auc_score(y_true=train['y'], y_score=train['y_pred'])
print(f"Train ROC AUC: {roc_auc:.3f}")

test = test.with_columns(y_pred = lr.predict_proba(test.drop('y'))[:, 1])
roc_auc = roc_auc_score(y_true=test['y'], y_score=test['y_pred'])
print(f"Test ROC AUC: {roc_auc:.3f}")

Train ROC AUC: 0.512
Test ROC AUC: 0.508


## Introducing interactions

By adding the interaction between $x_1$ and $x_2$, the model is able to *discover* the underlying function for data generation and reach perfect predictions.



In [5]:
train = train.with_columns(x1_x2 = pl.col('x1') * pl.col('x2'))
test = test.with_columns(x1_x2 = pl.col('x1') * pl.col('x2'))

lr = LogisticRegression().fit(train[['x1', 'x2', 'x1_x2']], train['y'])

train = train.with_columns(y_pred = lr.predict_proba(train[['x1', 'x2', 'x1_x2']])[:, 1])
roc_auc = roc_auc_score(y_true=train['y'], y_score=train['y_pred'])
print(f"Train ROC AUC: {roc_auc:.3f}")

test = test.with_columns(y_pred = lr.predict_proba(test[['x1', 'x2', 'x1_x2']])[:, 1])
roc_auc = roc_auc_score(y_true=test['y'], y_score=test['y_pred'])
print(f"Test ROC AUC: {roc_auc:.3f}")

Train ROC AUC: 1.000
Test ROC AUC: 1.000


--- 

**However**, if we have more than just two predictors, that would be a difficult task.
An alternative, is to use a more complex model. Here I show it with a simple decision tree. In real world data it might not be enough.

In [6]:
clf = DecisionTreeClassifier().fit(train[['x1', 'x2']], train['y'])

train = train.with_columns(y_pred_tree = clf.predict_proba(train[['x1', 'x2']])[:, 1])
roc_auc = roc_auc_score(y_true=train['y'], y_score=train['y_pred'])
print(f"Train ROC AUC: {roc_auc:.3f}")

test = test.with_columns(y_pred_tree = clf.predict_proba(test[['x1', 'x2']])[:, 1])
roc_auc = roc_auc_score(y_true=test['y'], y_score=test['y_pred'])
print(f"Test ROC AUC: {roc_auc:.3f}")

Train ROC AUC: 1.000
Test ROC AUC: 1.000
